# 快速参考

参考：[Docker 小技巧](https://www.mianshigee.com/project/jinfagang-alfred)

````{admonition} Docker 命令行的基本使用
:class: tip
```bash
docker + 管理对象（比如容器，镜像） + 具体操作（比如创建，启动，停止，删除）
```

```{admonition} 例子
- `docker image pull nginx` 拉取 `nginx` 的 docker 镜像
- `docker container stop web` 停止 `web` 的 docker 容器
```
````

```{include} quickref.txt
```

## 镜像管理和发布

```{topic} 镜像获取
- `pull`：从 registry 拉取
- `build`：从 Dockerfile 构建
- `load`：从本地获取
```

### 镜像基本操作

查看基本功能：

In [2]:
!docker image


Usage:  docker image COMMAND

Manage images

Commands:
  build       Build an image from a Dockerfile
  history     Show the history of an image
  import      Import the contents from a tarball to create a filesystem image
  inspect     Display detailed information on one or more images
  load        Load an image from a tar archive or STDIN
  ls          List images
  prune       Remove unused images
  pull        Pull an image or a repository from a registry
  push        Push an image or a repository to a registry
  rm          Remove one or more images
  save        Save one or more images to a tar archive (streamed to STDOUT by default)
  tag         Create a tag TARGET_IMAGE that refers to SOURCE_IMAGE

Run 'docker image COMMAND --help' for more information on a command.


```{topic} 镜像拉取
默认从 Docker Hub 拉取，如果不指定版本，会拉取最新版。
```

比如：

In [7]:
!docker pull nginx

Using default tag: latest
latest: Pulling from library/nginx
Digest: sha256:790711e34858c9b0741edffef6ed3d8199d8faa33f2870dea5db70f16384df79
Status: Image is up to date for nginx:latest
docker.io/library/nginx:latest


指定版本：

In [5]:
!docker pull nginx:1.20.0

1.20.0: Pulling from library/nginx

2152171a: Pulling fs layer 
15a5cec8: Pulling fs layer 
b026b9ce: Pulling fs layer 
dc384fb3: Pulling fs layer 
372d6dee: Pulling fs layer 
Digest: sha256:ea4560b87ff03479670d15df426f7d02e30cb6340dcd3004cdfc048d6a1d54b4
Status: Downloaded newer image for nginx:1.20.0
docker.io/library/nginx:1.20.0


从 Quay 上拉取镜像：

In [6]:
!docker pull quay.io/bitnami/nginx

Using default tag: latest
Error response from daemon: unauthorized: access to the requested resource is not authorized


````{topic} 镜像的查看

比如：
```bash
$ docker image ls
```
```
REPOSITORY              TAG       IMAGE ID       CREATED       SIZE
quay.io/bitnami/nginx   latest    0922eabe1625   6 hours ago   89.3MB
nginx                   1.20.0    7ab27dbbfbdf   10 days ago   133MB 
```  

查看虚悬镜像：
```bash
!docker image ls -f dangling=true
```
````

````{topic} 镜像的删除
```bash
$ docker image rm 0922eabe1625
```
```
Untagged: quay.io/bitnami/nginx:latest
Untagged: quay.io/bitnami/nginx@sha256:d143befa04e503472603190da62db157383797d281fb04e6a72c85b48e0b3239
Deleted: sha256:0922eabe16250e2f4711146e31b7aac0e547f52daa6cf01c9d00cf64d49c68c8
Deleted: sha256:5eee4ed0f6b242e2c6e4f4066c7aca26bf9b3b021b511b56a0dadd52610606bd
Deleted: sha256:472a75325eda417558f9100ff8b4a97f4a5e8586a14eb9c8fc12f944b26a21f8
Deleted: sha256:cdcb5872f8a64a0b5839711fcd2a87ba05795e5bf6a70ba9510b8066cdd25e76
Deleted: sha256:e0f1b7345a521469bbeb7ec53ef98227bd38c87efa19855c5ba0db0ac25c8e83
Deleted: sha256:11b9c2261cfc687fba8d300b83434854cc01e91a2f8b1c21dadd937e59290c99
Deleted: sha256:4819311ec2867ad82d017253500be1148fc335ad13b6c1eb6875154da582fcf2
Deleted: sha256:784480add553b8e8d5ee1bbd229ed8be92099e5fb61009ed7398b93d5705a560
Deleted: sha256:e0c520d1a43832d5d2b1028e3f57047f9d9f71078c0187f4bb05e6a6a572993d
Deleted: sha256:94d5b1d6c9e31de42ce58b8ce51eb6fb5292ec889a6d95763ad2905330b92762
Deleted: sha256:95deba55c490bbb8de44551d3e6a89704758c93ba8503a593cb7c07dfbae0058
Deleted: sha256:1ad1d903ef1def850cd44e2010b46542196e5f91e53317dbdb2c1eedfc2d770c
```
````

````{topic} 镜像导入
```bash
$ docker image load -i .\nginx.image
```
```
1839f9962bd8: Loading layer [==================================================>]   64.8MB/64.8MB
a2f4f809e04e: Loading layer [==================================================>]  3.072kB/3.072kB
9b63e6289fbe: Loading layer [==================================================>]  4.096kB/4.096kB
f7141923aaa3: Loading layer [==================================================>]  3.584kB/3.584kB
272bc57d3405: Loading layer [==================================================>]  7.168kB/7.168kB
Loaded image: nginx:1.20.0
```
````

````{topic} 镜像保存
```bash
$ docker image save nginx:1.20.0 -o nginx.image
```
````

### Dockerfile 基本结构

Docker 可以通过从 `Dockerfile` 中读取指令来自动构建映像。Dockerfile 是文本文档，它包含用户可以在命令行上调用的所有命令来组装镜像。

比如在 Ubuntu 系统下创建简单的 Python 环境：

```Dockerfile
FROM ubuntu:20.04
RUN apt-get update && \
    DEBIAN_FRONTEND=noninteractive apt-get install --no-install-recommends -y python3.9 python3-pip python3.9-dev
ADD hello.py /
CMD ["python3", "/hello.py"]
```

## scratch 镜像

Scratch 是空白 Docker 镜像。

```{rubric} 通过 scratch 来构建基础镜像。
```

::::{tab-set}

:::{tab-item} hello.c
```
#include <stdio.h>
int main()
{
    printf("hello docker\n");
}
```
:::
::::

```{rubric} 编译成二进制文件
```
```bash
$ gcc --static -o hello hello.c
$ ./hello
hello docker
$
```

::::{tab-set}

:::{tab-item} Dockerfile
```
FROM scratch
ADD hello /
CMD ["/hello"]
```
:::
::::

```{rubric} 构建
```

```bash
$ docker build -t hello .
$ docker image ls
REPOSITORY   TAG       IMAGE ID       CREATED          SIZE
hello        latest    2936e77a9daa   40 minutes ago   872kB
```

```{rubric} 运行
```
```bash
$ docker container run -it hello
hello docker
```

## Docker 数据存储

默认情况下，在运行中的容器里创建的文件，被保存在可写的容器层：

- 如果容器被删除了，则数据也没有了
- 这个可写的容器层是和特定的容器绑定的，也就是这些数据无法方便的和其它容器共享

Docker 主要提供了两种方式做数据的持久化

- Data Volume：由 Docker 管理（`/var/lib/docker/volumes/` Linux）， 持久化数据的最好方式
- Bind Mount，由用户指定存储的数据具体 mount 在系统什么位置
